# Perplexity playbook

<p>
Mal Minhas, v0.7<br>
25.01.25
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.3</b>: 15.01.24. First version</li>
<li><b>v0.4</b>: 02.06.24. Updated for latest Perplexity API</li>
<li><b>v0.5</b>: 01.08.24. Updated again for latest Perplexity API and llama 3.1 support</li>
<li><b>v0.6</b>: 09.11.24. Updated to include support for retrieving citations</li>
<li><b>v0.7</b>: 25.01.25. Updated to latest Perplexity models and added streaming support</li>
</ul>
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an OpenAI-compliant API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api` per the documetnation [here](https://docs.perplexity.ai/).  Perplexity regularly update the model support for this API and at the time of writing the latest and greatest supported model is `llama-3.1`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/guides/model-cards):
```
Model	Context Length	Model Type
sonar-pro	200k	Chat Completion
sonar	127k	Chat Completion
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import display, Markdown

def renderTextAsMarkdown(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get('PERPLEXITY_API_KEY')
DEFAULT_MODEL = 'sonar'
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = 'Be precise and concise.'
MID_LEVEL_DETAIL = 'Explain in a few sentences.'
GREAT_DETAIL = 'Explain in a lot of detail.'
POLITE_DETAIL = 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return r.json()

def renderResponse(d):
    return renderTextAsMarkdown(d.get("choices")[0].get("message").get("content"))

def getModel(d):
    return renderTextAsMarkdown(d.get("model"))
    
def getCitations(d):
    def formatReferences(url_list):
        return [f"[{i + 1}]: {url}" for i, url in enumerate(url_list)]
    for ref in formatReferences(d.get("citations")):
        print(ref)

Let's test the function on an example query using the default `llama-3.1-sonar-small-128k-online` model and then `llama-3.1-8b-instruct`.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
d = getPerplexityResponse(prompt)
renderResponse(d)

To get your broken washing machine mended in Southampton, you should contact a professional appliance repair service. Trades such as appliance repair technicians or engineers are the best to call for this task. Companies like NAC and Go Assist offer reliable and fast washing machine repair services in Southampton, providing same-day or next-day appointments and covering a wide range of makes and models[1][2]. You can also consider contacting Harrap Domestic Appliances for comprehensive repair services[3].

CPU times: user 14.1 ms, sys: 7.09 ms, total: 21.2 ms
Wall time: 3.39 s


As of 09.11.24, Perplexity are offering citations as part of the API response.  We can pull them out as follows.  The URLs all work and point to valid companies that could help with this job:

In [4]:
getCitations(d)

[1]: https://nacrepair.co.uk/appliance-repairs/washing-machine-repair/washing-machine-repair-southampton/
[2]: https://go-assist.co.uk/appliance-repairs/washing-machine-repairs/washing-machine-repair-southampton
[3]: https://www.harrapdomesticappliances.co.uk/p/appliances-repairs-and-spares
[4]: https://www.southampton-appliance-repairs.co.uk/help/terms-conditions
[5]: https://nacrepair.co.uk/appliance-repair/appliance-repair-southampton/


In [5]:
%%time

renderResponse(getPerplexityResponse(prompt, model='sonar'))

To get your washing machine repaired in Southampton, you should contact a professional appliance repair service. Trades such as appliance repair engineers or technicians are the ones you need to call. Companies like NAC, Go Assist, and Hedge End Domestics offer reliable and fast washing machine repair services in Southampton, often providing same or next-day visits[1][2][3]. They can diagnose and fix various issues with different types of washing machines.

CPU times: user 11.9 ms, sys: 5.71 ms, total: 17.6 ms
Wall time: 3.25 s


In [6]:
d = getPerplexityResponse(prompt, model='sonar-pro', direction=POLITE_DETAIL)
renderResponse(d)
getCitations(d)

If you have a broken washing machine in Southampton, UK, you have several options to get it repaired. You'll want to call an appliance repair specialist or a washing machine repair service. Here are some steps you can take:

1. Contact a local appliance repair service:
Several companies in Southampton offer washing machine repair services. For example, NAC Repair provides same-day or next-day washing machine repairs in Southampton[1][7]. They have a network of engineers serving the area and can often complete repairs on the spot.

2. Book a repair appointment:
Many companies allow you to book repairs online or over the phone. NAC Repair, for instance, offers an online booking system for easy scheduling[7].

3. Consider the repair costs:
Most reputable companies will provide a free quote before starting any work. Some, like Go Assist, offer competitive and transparent pricing without hidden fees[3].

4. Check for guarantees:
Look for companies that offer guarantees on their work. For example, NAC backs their repairs with a guarantee, promising to fix any reoccurring faults free of charge[1].

5. Ensure the repairer is experienced with your machine:
Reputable repair services like NAC and Go Assist can handle all makes and models of washing machines, including popular brands like Bosch, Samsung, Hotpoint, and Indesit[1][3].

6. Ask about same-day service:
If your need is urgent, many companies offer same-day or next-day service. Go Assist, for instance, provides quick service to minimize appliance downtime[3].

7. Check for additional services:
Some companies, like Harrap Domestic Appliances, offer advice on upgrades if your washing machine is beyond repair[4].

Remember to choose a licensed and insured repair service for your safety and peace of mind. If you're unsure about a company's credentials, don't hesitate to ask for proof of their qualifications and insurance.

[1]: https://nacrepair.co.uk/appliance-repairs/washing-machine-repair/washing-machine-repair-southampton/
[2]: https://hemiappliances.com/southampton-appliance-repair-pa-18966-southampton-pa-appliance-repair-southampton-pennsylvania-18966-southampton-appliance-repair-pennsylvania-southampton-18966-pa/
[3]: https://go-assist.co.uk/appliance-repairs/washing-machine-repairs/washing-machine-repair-southampton
[4]: https://www.harrapdomesticappliances.co.uk/p/appliances-repairs-and-spares
[5]: https://appliancerepairsouthampton.ny-biz.co
[6]: https://www.southampton-appliance-repairs.co.uk/help/terms-conditions
[7]: https://nacrepair.co.uk/appliance-repair/appliance-repair-southampton/
[8]: https://www.southamptonnyappliancerepair.com
[9]: https://www.advancedappliancecare.co.uk/find-appliance-repair-specialist-can-trust/
[10]: https://www.aaqualitydomesticappliances.co.uk


In [7]:
%%time

renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=POLITE_DETAIL))

If you have a broken washing machine and live in Southampton, UK, there are several options to get it mended. Here are a few steps you can take:

1. **Contact Local Appliance Repair Services:**
   - **NAC Repair:** NAC offers same-day or next-day washing machine repairs in Southampton. They have a network of engineers who can diagnose and fix various types of washing machines, including front-loading, top-loading, semi-automatic, and fully automatic models[1].
   - **Go Assist:** Go Assist provides trusted washing machine repair services in and around Southampton. They offer same or next-day service, competitive pricing, and a repair guarantee. Their engineers are experienced in repairing all makes and models of washing machines[3].
   - **Harrap Domestic Appliances:** Harrap offers comprehensive and affordable washing machine repair services in Southampton. Their highly skilled team can diagnose and repair a wide range of washing machine issues[4].

2. **Book a Repair Online:**
   - You can book your repair online with NAC or Go Assist to ensure quick and reliable service.

3. **Call for Immediate Assistance:**
   - If you need immediate assistance, you can call NAC directly for same-day or next-day repairs, or contact Go Assist for their on-demand service.

4. **Check Reviews and Guarantees:**
   - Before choosing a service, check reviews and guarantees offered by each company to ensure you're getting reliable and high-quality service.

5. **Additional Options:**
   - If you prefer to contact a local service directly, you can also look up other local appliance repair services in Southampton, such as Raynor Appliance Service, which offers a one-year guarantee for parts and labor[5].

By following these steps, you should be able to find a reliable and efficient solution to get your washing machine mended quickly.

CPU times: user 12.8 ms, sys: 8.34 ms, total: 21.2 ms
Wall time: 3.14 s


Let's see how it handles a query with a more detailed response:

In [8]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=GREAT_DETAIL))

To fix a broken garden gate that is no longer closing properly, you should call a gate repair technician. These professionals specialize in diagnosing and repairing issues with gates, including problems with hinges, latches, and the overall alignment of the gate.

Here are some key points to consider:

1. **Specialized Tools and Expertise**: A gate repair technician has the right tools and expertise to assess and fix the problem accurately. They can identify issues that might not be immediately apparent, such as worn-out hinges or misaligned posts, which can prevent the gate from closing properly[1][2].

2. **Common Repairs**: The most common repairs for a sagging or misaligned gate include checking and adjusting the hinge system, ensuring that the gate is properly aligned, and possibly replacing damaged parts like hinges or latches[1][2][4].

3. **DIY vs. Professional**: While some DIY guides are available for fixing a sagging gate, hiring a professional can ensure that the job is done correctly and safely. Professionals can also provide long-term solutions by addressing underlying issues that might cause future problems[2][4].

4. **Cost Estimates**: It's a good idea to contact several gate repair professionals to get accurate cost estimates for your specific repair needs. The cost can vary based on the type of repair required, the materials needed, and the complexity of the job[1].

By calling a gate repair technician, you can ensure that your garden gate is fixed efficiently and effectively, restoring its functionality and ensuring your garden remains secure and accessible.

CPU times: user 11.3 ms, sys: 7.96 ms, total: 19.2 ms
Wall time: 2.79 s


## 3. Streaming interface

Leveraging the OpenAI interface:

In [9]:
import asyncio
from openai import OpenAI
from typing import List

references = []

async def getPerplexityResponseStreaming(question:str, references:List, direction:str=MID_LEVEL_DETAIL) -> str:
    messages = [
        {
            "role": "system",
            "content": (direction),
        },
        {   
            "role": "user",
            "content": (question),
        },
    ]
    client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")
    # chat completion with streaming
    response_stream = client.chat.completions.create(
        model="sonar-pro",
        messages=messages,
        stream=True,
    )
    for response in response_stream:
        if response.citations:
            references += response.citations
        yield response.choices[0].delta.content

async def processStreamingResponse(prompt, references):
    async for ref in getPerplexityResponseStreaming(prompt, references):
        print(ref, end="")  # Print each result as soon as it's yielded without adding newlines

# Jupyter-specific way to run asyncio in a cell
prompt = "How many stars are there in our galaxy?"

# Use an event loop in Jupyter to run the async function
await processStreamingResponse(prompt, references)
for i,ref in enumerate(references):
    print(f"{i}: {ref}")

The Milky Way galaxy contains approximately 100-400 billion stars[1][3][6]. This estimate is based on calculations of the galaxy's mass and the percentage of that mass made up of stars[6]. However, determining an exact number is challenging due to the difficulty in counting stars individually across such vast distances[3]. The range in estimates stems from uncertainties in factors like the average mass of stars and the total mass of the galaxy[6]. While this number is enormous, it's worth noting that the Milky Way is just one of an estimated 2 trillion galaxies in the observable universe[4].0: https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/
1: https://www.littlepassports.com/blog/space/how-many-stars-are-in-the-universe/
2: https://www.space.com/25959-how-many-stars-are-in-the-milky-way.html
3: https://bigthink.com/starts-with-a-bang/how-many-stars/
4: https://www.youtube.com/watch?v=Py2nZYmvTKg
5: https://asd.gsfc.nasa.gov/blueshift/index.php/2015/0